In [1]:
import torch

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


/home/nahid/anaconda3/envs/pytorch/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import sys,os, pickle

import torch.nn as nn
from transformers import BertModel, BertTokenizer
from transformers import BertForMaskedLM
from torch.utils.data import DataLoader, Dataset

from transformers import AdamW
from tqdm import tqdm  # for our progress bar

from sklearn.metrics import precision_recall_fscore_support , accuracy_score
import numpy as np

with (open('DFGDATA.pkl' , "rb")) as openfile:
    all_input_list , all_label_list = pickle.load(openfile)


# input_text = ["I love apples.", "She hates oranges.", "I enjoy reading books.", "He likes playing football."]

# labels_task1 = [0, 1,0,1]  # Example task 1 labels
# labels_task2 = [2, 3,4,5]  # Example task 2 labels
BATCH_SIZE = 4

input_text = all_input_list#[:180]
labels_dfg = all_label_list#[:180]

labels_dfg =  torch.LongTensor(labels_dfg)

In [3]:
tokenizer  = BertTokenizer.from_pretrained("./multytask-tokenizer")
VOCAB_SIZE = tokenizer.vocab_size

print('VOCAB SIZE : ', (tokenizer.vocab_size))


VOCAB SIZE :  10000


In [4]:
# from transformers import DataCollatorForLanguageModeling

# tokenizer.eos_token = "<EOI>"

# tokenizer.pad_token = tokenizer.eos_token
# data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [5]:
# data_collator(input_text_task1)

In [6]:
#TODO fix inputs = tokenizer(history, next_instruction, return_tensors='pt', 
#                    max_length=64, truncation=True, padding='max_length')

# Tokenize input text
inputs = tokenizer(input_text, padding=True, truncation=True, return_tensors='pt',)
# encoded_inputs_task2 = tokenizer(input_text_task2, padding=True, truncation=True, return_tensors='pt')

inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [7]:
tokenizer('LOOK')

{'input_ids': [2, 100, 3], 'token_type_ids': [0, 0, 0], 'attention_mask': [1, 1, 1]}

In [8]:
inputs['labels_dfg'] = labels_dfg.to(device)

In [9]:
#MLM
inputs['labels'] = inputs.input_ids.detach().clone()


# create random array of floats with equal dimensions to input_ids tensor
rand = torch.rand(inputs.input_ids.shape)
# create mask array
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * \
        (inputs.input_ids != 102) * (inputs.input_ids != 0)

#TODO FIX 101, 102

# mask_arr[0]

In [10]:
selection = []

for i in range(inputs.input_ids.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )
# selection[0]

In [11]:
# inputs.input_ids

In [12]:
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels_dfg', 'labels'])

In [13]:
print(inputs.keys())
print(inputs.token_type_ids)
inputs.token_type_ids[0][0] =1
print(inputs.token_type_ids)
#TODO set token type ids

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels_dfg', 'labels'])
tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
tensor([[1, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])


In [14]:
class BinaryDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [15]:
dataset = BinaryDataset(inputs)
train_size = int(0.8 * len(dataset))
validation_size = len(dataset) - train_size

train_dataset, validation_dataset = torch.utils.data.random_split(dataset, [train_size, validation_size])
train_loader      = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
validation_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [16]:
# global_task2_metrices, global_task1_metrices ,v_global_task1_metrices, v_global_task2_metrices

from numpy import *
import math
import matplotlib.pyplot as plt


def plot_graph(task1_metrices, task2_metrices , v_task1_metrices, v_task2_metrices, label = "Graph" ):
    
    plt.ioff()

    font_size = 10
    x_labels = [ i for i in range(len(task1_metrices)) ]
    
    task1_f1 = [ i['f1'] for i in task1_metrices ]
    task2_f1 = [ i['f1'] for i in task2_metrices ]
    
    v_task1_f1 = [ i['f1'] for i in v_task1_metrices ]
    v_task2_f1 = [ i['f1'] for i in v_task2_metrices ]
    

    plt.ylabel(' F1 ',fontsize=font_size)
    plt.plot(x_labels, task1_f1 , 'r') 
    plt.plot(x_labels, task2_f1 , 'b') 
    
    plt.plot(x_labels, v_task1_f1 , 'r' , linestyle = '--') 
    plt.plot(x_labels, v_task2_f1 , 'b' , linestyle = '--') 
    
    plt.xlabel("Epoch", fontsize=font_size)
    plt.title(label,fontsize=font_size)
    plt.legend(['MLM Train', 'DFG Train', 'MLM Val', 'DFG Val'], loc='upper left')
    
    plt.savefig('./output/'+label+'_f1.pdf')
    plt.close()
#     plt.show()
    
    ################################
    ################# LOSS #########
    ################################
    
    task1_loss = [ i['loss'] for i in task1_metrices ]
    task2_loss = [ i['loss'] for i in task2_metrices ]
    
    v_task1_loss = [ i['loss'] for i in v_task1_metrices ]
    v_task2_loss = [ i['loss'] for i in v_task2_metrices ]
    

    plt.ylabel(' LOSS ',fontsize=font_size)
    plt.plot(x_labels, task1_loss , 'r') 
    plt.plot(x_labels, task2_loss , 'b') 
    
    plt.plot(x_labels, v_task1_loss , 'r' , linestyle = '--') 
    plt.plot(x_labels, v_task2_loss , 'b' , linestyle = '--') 
    
    plt.xlabel("Epoch", fontsize=font_size)
    plt.title(label,fontsize=font_size)
    plt.legend(['MLM Train', 'DFG Train', 'MLM Val', 'DFG Val'], loc='upper left')
    
    plt.savefig('./output/'+label+'_loss.pdf')
    plt.close()
#     plt.show()
    
    

In [17]:

# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# Define the multitask BERT model

hidden_size = BertModel.from_pretrained('bert-base-uncased').config.hidden_size
# vocab_size = bert_model.config.vocab_size
class MultitaskBert(nn.Module):
    def __init__(self):
        super(MultitaskBert, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased') #BertModel.from_pretrained('bert-base-uncased')
        self.fc = nn.Linear(  hidden_size , VOCAB_SIZE)
        self.dfg_classifier = nn.Linear(hidden_size, 2) #todo was 768 num_classes_task2



    def forward(self, input_ids, attention_mask ):
        bert_outputs = self.bert(input_ids, attention_mask=attention_mask)

        hidden_states = bert_outputs.last_hidden_state
        logits_mlm = self.fc(hidden_states)
        
        bert_pooled_output = bert_outputs.pooler_output
        logits_dfg = self.dfg_classifier(bert_pooled_output)
        
        
        return logits_mlm,   logits_dfg




Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [18]:
# Create model instance
model = MultitaskBert()  # Example number of classes for each task
model.to(device)

# initialize optimizer
optim = AdamW( model.parameters() , lr=5e-6)
criterion = nn.CrossEntropyLoss()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [19]:
def training_loop(model ,data_loop, is_training = False):
    
    task1_prediction_s, task1_ground_truth_s = [], []
    task1_losses = []
    
    task2_prediction_s, task2_ground_truth_s = [], []
    task2_losses = []
    
    for N,batch in enumerate(data_loop):


        # Forward pass
        if is_training == True:
            optim.zero_grad()

        logits_task1,   logits_task2 = model(batch['input_ids'].to(device), batch['attention_mask'].to(device))


        #Task 1
        loss_task1 = criterion(logits_task1.view(-1, VOCAB_SIZE), batch['labels'].to(device).view(-1))
        task1_losses.append(loss_task1.item())
        predictions_task1 = torch.argmax(logits_task1, dim=2)


        task1_prediction_s.extend(predictions_task1.detach().cpu().numpy().flatten())
        task1_ground_truth_s.extend(batch['labels'].detach().cpu().numpy().flatten())

        #Task2
        loss_task2 = criterion(logits_task2, batch['labels_dfg'].to(device))
        task2_losses.append(loss_task2.item())
        predictions_task2 = torch.argmax(logits_task2, dim=1)


        task2_prediction_s.extend(predictions_task2.detach().cpu().numpy().flatten())   
        task2_ground_truth_s.extend(batch['labels_dfg'].detach().cpu().numpy().flatten())

        if is_training == True:
            (loss_task1+loss_task2).backward()
            optim.step()


        # print relevant info to progress bar
        data_loop.set_description(f'Epoch {ecpoch}')
        data_loop.set_postfix(loss=(loss_task1+loss_task2).item())

    ###### Training Scores
    task1_accuracy = (accuracy_score(task1_ground_truth_s, task1_prediction_s))    
    task1_precision, task1_recall, task1_f1, _ = precision_recall_fscore_support(task1_ground_truth_s,task1_prediction_s,average='weighted')
    task1_metrices = {'accuracy':task1_accuracy ,
                      'precision':task1_precision, 
                      'recall':task1_recall, 
                      'f1':task1_f1,
                      'loss': (sum(task1_losses) / len(task1_losses))}
    
    task2_accuracy = (accuracy_score(task2_ground_truth_s, task2_prediction_s))   
    task2_precision, task2_recall, task2_f1, _ = precision_recall_fscore_support(task2_ground_truth_s,task2_prediction_s, average='binary')
    task2_metrices = {'accuracy':task2_accuracy ,
                      'precision':task2_precision, 
                      'recall':task2_recall, 
                      'f1':task2_f1,
                      'loss': (sum(task2_losses) / len(task2_losses))}

    return task1_metrices, task2_metrices
    

In [ ]:
EPOCHS = 10


global_task2_metrices = []
global_task1_metrices = []

v_global_task1_metrices = []
v_global_task2_metrices = []

for ecpoch in range(EPOCHS):
    
    train_loop = tqdm(train_loader, leave=True)
    model.train()
    task1_metrices , task2_metrices = training_loop(model ,train_loop, is_training = True)
    global_task1_metrices.append(task1_metrices)
    global_task2_metrices.append(task2_metrices)
    
    with torch.no_grad():
        model.eval()
        validation_loop = tqdm(validation_loader, leave=True)
        v_task1_metrices , v_task2_metrices = training_loop(model ,validation_loop, is_training = False)
        v_global_task1_metrices.append(v_task1_metrices)
        v_global_task2_metrices.append(v_task2_metrices)
    
    plot_graph(global_task1_metrices, global_task2_metrices ,v_global_task1_metrices, v_global_task2_metrices)

  0%|                                                  | 0/7706 [00:00<?, ?it/s]/tmp/ipykernel_1042747/3277124069.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
Epoch 0:   7%|█▎                | 541/7706 [04:38<1:04:31,  1.85it/s, loss=2.79]

In [ ]:
1